## Motivation

In this note, we exame the idea declared in section 1.5.5 on MNIST data.

In [1]:
import numpy as np
import tensorflow as tf
from keras.losses import MSE
from sklearn.metrics import accuracy_score

from utils import get_gradient_loss_fn

tf.random.set_seed(42)

2024-03-19 18:27:51.090514: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## The MNIST Data

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32').reshape([-1, 28*28]) / 255.0
x_test = x_test.astype('float32').reshape([-1, 28*28]) / 255.0
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

## Train a Model with Gradient Loss

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, 'relu'),
    tf.keras.layers.Dense(128, 'relu'),
    tf.keras.layers.Dense(10, 'softmax')
])

get_gradient_loss = get_gradient_loss_fn(
    lambda inputs: MSE(inputs[1], model(inputs[0]))
)

In [4]:
optimizer = tf.optimizers.Adam()

@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        loss = get_gradient_loss((x, y))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(zip(grads, model.variables))
    return loss

In [5]:
def evaluate(model):
    return accuracy_score(y_test, tf.argmax(model(x_test), axis=1))

In [6]:
ds = tf.data.Dataset.from_tensor_slices(
    (x_train, tf.one_hot(y_train, 10))
)
ds = ds.batch(100)

In [7]:
for epoch in range(20):
    for x, y in ds:
        loss = train_step(x, y)
    print(epoch, loss)

0 tf.Tensor(0.00027563714, shape=(), dtype=float32)
1 tf.Tensor(0.00022184353, shape=(), dtype=float32)
2 tf.Tensor(0.00019450903, shape=(), dtype=float32)
3 tf.Tensor(0.00013707536, shape=(), dtype=float32)
4 tf.Tensor(0.00012243888, shape=(), dtype=float32)
5 tf.Tensor(0.00013177721, shape=(), dtype=float32)
6 tf.Tensor(8.567769e-05, shape=(), dtype=float32)
7 tf.Tensor(0.00014986162, shape=(), dtype=float32)
8 tf.Tensor(0.000103624225, shape=(), dtype=float32)
9 tf.Tensor(8.7943845e-05, shape=(), dtype=float32)
10 tf.Tensor(8.1980375e-05, shape=(), dtype=float32)
11 tf.Tensor(8.081834e-05, shape=(), dtype=float32)
12 tf.Tensor(8.0793616e-05, shape=(), dtype=float32)
13 tf.Tensor(8.018364e-05, shape=(), dtype=float32)
14 tf.Tensor(8.086579e-05, shape=(), dtype=float32)
15 tf.Tensor(8.0154576e-05, shape=(), dtype=float32)
16 tf.Tensor(8.0194935e-05, shape=(), dtype=float32)
17 tf.Tensor(8.8471796e-05, shape=(), dtype=float32)
18 tf.Tensor(0.00012438378, shape=(), dtype=float32)
19 tf.

In [8]:
evaluate(model)

0.9801

## Baseline Model

In [9]:
baseline_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, 'softmax')
])

In [10]:
baseline_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
)

In [11]:
baseline_model.fit(
    x_train, y_train,
    epochs=20,
    validation_data=(x_test, y_test),
)

Epoch 1/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2309 - accuracy: 0.9328 - val_loss: 0.1219 - val_accuracy: 0.9630
Epoch 2/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0966 - accuracy: 0.9699 - val_loss: 0.1253 - val_accuracy: 0.9600
Epoch 3/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0676 - accuracy: 0.9782 - val_loss: 0.0735 - val_accuracy: 0.9766
Epoch 4/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0504 - accuracy: 0.9831 - val_loss: 0.0761 - val_accuracy: 0.9772
Epoch 5/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0396 - accuracy: 0.9874 - val_loss: 0.0945 - val_accuracy: 0.9728
Epoch 6/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0328 - accuracy: 0.9891 - val_loss: 0.0854 - val_accuracy: 0.9762
Epoch 7/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0263 - accuracy: 0.9916 - val_loss: 0.0841 - val_accuracy:

In [12]:
evaluate(baseline_model)

0.9792

## Model Robustness

Now, we compare the robustness of the model and the baseline. To do so, we add Gaussian noise to the test data and check the accuracy.

In [25]:
stddev = 3e-1
noise = tf.random.normal(tf.shape(x_test)) * stddev

In [26]:
def evaluate_robustness(model):
    accuracy = accuracy_score(
        y_test, tf.argmax(model(x_test), axis=1))
    noised_accuracy = accuracy_score(
        y_test, tf.argmax(model(x_test+noise), axis=1))
    print(f'Accuracy: {accuracy} -> {noised_accuracy}')

In [27]:
evaluate_robustness(model)

Accuracy: 0.9801 -> 0.9464


In [28]:
evaluate_robustness(baseline_model)

Accuracy: 0.9792 -> 0.9098


## Conclusion

By simply using the "gradient loss", we obtain a result that is the same as the baseline. But the robustness is greatly out-performs the baseline.